# Домашнее задание к лекции «Основы парсинга и работы с API»

In [206]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

## Задание 1

###### Обязательная часть

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

###### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [207]:
res = requests.get('https://habr.com/ru/all/')

In [210]:
KEYWORDS = ['рассказывает', 'Алгоритмы']

posts = BeautifulSoup(res.text, 'html.parser').find_all('article', class_='post')

habr_articles = pd.DataFrame()

    
for post in posts:    
    time.sleep(0.2)    
    
    post_id = post.parent.attrs.get('id')
    
    if not post_id:
        continue
    
    title = post.find(class_='post__title_link').text
    href = post.find(class_='post__title_link')['href']
        
    if href: 
        content = BeautifulSoup(requests.get(href).text, 'html.parser')
        post_text = post.text.lower()
        content_text = content.find(class_='post_full').text.lower() 
        
        if any([word.lower() in content_text for word in KEYWORDS]):
            
            text = content.find(class_='post__body').get_text(strip=True)
            date = content.find(class_='post__time').get('data-time_published')
            
            row = {'date': date, 'title': title, 'href': href, 'text': text}
            
            habr_articles = pd.concat([habr_articles, pd.DataFrame([row])])  
        

habr_articles

,date,title,href,text
0,2021-01-16T11:52Z,Как стать тестировщиком или что я узнал за вре...,https://habr.com/ru/post/537628/,Привет Хабр! В этот пост я хочу вынести опыт н...
0,2021-01-16T10:26Z,Крутые часы на адресных диодах (часть 3),https://habr.com/ru/post/537622/,"Часть 3. Прошивка, настройка и алгоритмы работ..."


### Задание 2

###### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [211]:
import json

EMAIL = ['illonnavins@gmail.com', 'vinspetro@rambler.ru']
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
HEADERS = {
    'Vaar-Version': '0',    
    'Vaar-Header-App-Product': 'hackcheck-web-avast'
}

params = {
    'emailAddresses': EMAIL
}

request = requests.post(URL, data=json.dumps(params), headers=HEADERS)

In [212]:
# Меня смущают 2 вложенных цикла, есть способ проще сформировать требуемый дата фрейм?

avast_list = pd.DataFrame()

data = request.json()

for info in data['data'].values():    
    
    for email, breaches in info.items(): 
        
        for breach in breaches: 
            
            breach_id = str(breach['breachId'])      
            breach_info = data['breaches'][breach_id]
            
            row = {
                'email': email, 
                'date': breach_info['publishDate'], 
                'site': breach_info['site'], 
                'description': breach_info['description']
            }
            
            avast_list = pd.concat([avast_list, pd.DataFrame([row])])         

    
avast_list

,email,date,site,description
0,vinspetro@rambler.ru,2019-03-20T00:00:00Z,bookmate.com,"In July 2018, Bookmate was allegedly breached...."
0,vinspetro@rambler.ru,2016-10-23T00:00:00Z,rambler.ru,"On February 17, 2012, Russian email provider R..."
0,vinspetro@rambler.ru,2020-12-03T00:00:00Z,sportzone.ru,"In November 2020, a collection of over 23,000 ..."
0,illonnavins@gmail.com,2020-12-10T00:00:00Z,bigpuzzle.ru,"In November 2020, a collection of over 23,000 ..."
0,illonnavins@gmail.com,2019-06-13T00:00:00Z,canva.com,"In May 2019, graphic-design site Canva's datab..."
0,illonnavins@gmail.com,2016-10-26T00:00:00Z,twitter.com,Login credentials for over 32 Million Twitter ...


### Задание 2

###### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>

In [213]:
VK_REQUEST = 'https://api.vk.com/method/wall.get'
GROUP = 'netology'
TOKEN = ''
VERSION = '5.103'

params = {
    'access_token': TOKEN,
    'v': VERSION,
    'domain': GROUP,
    'count': 50
}

netolgy_posts = requests.get(VK_REQUEST, params).json()

In [214]:
netolgy_df = pd.DataFrame()

for post in netolgy_posts['response']['items']:
    
    date = pd.to_datetime(post['date'], unit='s').strftime('%d.%m.%Y')

    row = {'date': date, 'text': post['text']}
            
    netolgy_df = pd.concat([netolgy_df, pd.DataFrame([row])])  
    

netolgy_df        

,date,text
0,11.01.2021,Врываемся в новый год с новыми знаниями: на эт...
0,16.01.2021,Подкасты любят за возможность проводить время ...
0,15.01.2021,"В эфире #пятничнаябеседка. Здесь мы обсуждаем,..."
0,15.01.2021,🔥 19 января стартует бесплатный онлайн-интенси...
0,14.01.2021,"Насмотренность — это визуальный опыт, который ..."
0,14.01.2021,Некоторые бренды можно узнать с первых слов: в...
0,13.01.2021,Разыгрываем 10 доступов к библиотеке видеокурс...
0,13.01.2021,🧘 26 января стартует бесплатный курс «Психолог...
0,12.01.2021,Изучение языка программирования можно сравнить...
0,12.01.2021,"Что делает копирайтер, как стать хорошим сммщи..."
